In [1]:
# Code extensively uses the kernel https://www.kaggle.com/bonhart/pytorch-cnn-from-scratch!

# Thank you https://www.kaggle.com/bonhart

In [2]:
# Libraries
import os
import numpy as np
!conda install --yes pandas 
import pandas as pd

import cv2
!conda install --yes seaborn 
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py38h578d9bd_1         145 KB  conda-forge
    conda-4.11.0               |   py38h578d9bd_0        16.9 MB  conda-forge
    pandas-1.3.5               |   py38h43a58ef_0        13.0 MB  conda-forge
    python-dateutil-2.8.2      |     pyhd8ed1ab_0         240 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        30.4 MB

The following NEW packages will be INSTALLED:

  pandas             conda-forge/linux-64::pandas-1.3.5-py38h43a58ef_0
  python-dateutil    conda-forge/noarch::python-dateutil-2.8.2-pyhd8ed1ab_0

The following pac

In [3]:
from glob import glob
imagePatches = glob('IDC_regular_ps50_idx5/IDC_regular_ps50_idx5/**/*.png', recursive=True)
len(imagePatches)

277524

In [4]:
imagePatches[0]

'IDC_regular_ps50_idx5/IDC_regular_ps50_idx5/12910/1/12910_idx5_x651_y201_class1.png'

In [5]:
import fnmatch
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)

In [6]:
torch.cuda.is_available()

True

In [7]:

z=[]
x=[]
X=[]
y=[]
Y=[]
ID=[]
cl=[]
labels=[]
Num_Folder=[]
matrix=[] #define empty matrix
i=0
for img in imagePatches:
    
    imagePatches1=imagePatches[i].split("/")
    #print(imagePatches1)
    imagePatches2=imagePatches1[4].split("_")
    #print(imagePatches2)
    x=imagePatches2[2].split("x")
    x=x[1]
    #print(x)
    y=imagePatches2[3].split("y")
    y=y[1]
    #print(y)
    labels=imagePatches2[4].split(".png")
    cl=labels[0]
   
    if cl=='class0':
        cl=0
    elif cl=='class1':
         cl=1
        
    ID=imagePatches2[1]
    Num_Folder=imagePatches2[0]
    images=imagePatches[i]
    #X=[Num_Folder,ID,x,y,images,cl]
    X=[images,cl]
     
    matrix.append(X) #add fully defined column into the row
    #z = []
    #if img in classZero:
        #z.append(0)
    #elif img in classOne:
       # z.append(1)
    i=i+1
#column_names=["Num_Folder","ID","x","y","images","labels"]
column_names=["images","labels"]
images_df = pd.DataFrame(matrix,columns=column_names)
images_df.head()
#images_df.to_csv(r'dataset.csv', index = False)



,images,labels
0,IDC_regular_ps50_idx5/IDC_regular_ps50_idx5/12...,1
1,IDC_regular_ps50_idx5/IDC_regular_ps50_idx5/12...,1
2,IDC_regular_ps50_idx5/IDC_regular_ps50_idx5/12...,1
3,IDC_regular_ps50_idx5/IDC_regular_ps50_idx5/12...,1
4,IDC_regular_ps50_idx5/IDC_regular_ps50_idx5/12...,1


In [8]:
images_df.groupby(['labels'])['labels'].count()

labels
0    198738
1     78786
Name: labels, dtype: int64

In [9]:
#Splitting data into train and val
train, val = train_test_split(images_df, stratify=images_df.labels, test_size=0.2)
len(train), len(val)

(222019, 55505)

In [10]:
class MyDataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values
        
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        
        image = cv2.imread(img_path)
        image = cv2.resize(image, (50,50))
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [11]:
## Parameters for model

# Hyper parameters
num_epochs = 15
num_classes = 2
batch_size = 128
learning_rate = 0.0002

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)

'Quadro M4000'

In [12]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = MyDataset(df_data=train, transform=trans_train)
dataset_valid = MyDataset(df_data=val,transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

In [13]:
class SimpleCNN(nn.Module):
    def __init__(self):
        # ancestor constructor call
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(7)
        self.fc = nn.Linear(512 * 1 * 1, 2) # !!!
        
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)))) # first convolutional layer then batchnorm, then activation then pooling layer.
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
        #print(x.shape) # lifehack to find out the correct dimension for the Linear Layer
        x = x.view(-1, 512 * 1 * 1) # !!!
        x = self.fc(x)
        return x

In [14]:
model = SimpleCNN().to(device)

In [15]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)

In [16]:
# Train the model
import gc
import time
x=[]
Epotch_stop=0
Int100_time=0
Int100_time1=np.zeros(1735)

total_step = len(loader_train)
for epoch in range(num_epochs):
    start= time.time()
    
    for i, (images, labels) in enumerate(loader_train):
        #print(images)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
       # gc.collect()
        #torch.cuda.empty_cache()
        loss = criterion(outputs, labels)
        
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
       
        if (i+1) % 100 == 0:
            stop= time.time()
            Int100_time=stop-start
            Int100_time1[i]=Int100_time1[i]+Int100_time           
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f},Int100_time:{}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(),Int100_time))
        elif (i+1) % 1735 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f},Epotch_time:{}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(),Int100_time1[i]))
    


Epoch [1/15], Step [100/1735], Loss: 0.3686,Int100_time:70.97502541542053
Epoch [1/15], Step [200/1735], Loss: 0.3551,Int100_time:139.48899722099304
Epoch [1/15], Step [300/1735], Loss: 0.3337,Int100_time:206.36218452453613
Epoch [1/15], Step [400/1735], Loss: 0.2863,Int100_time:271.8614764213562
Epoch [1/15], Step [500/1735], Loss: 0.3070,Int100_time:329.8052339553833
Epoch [1/15], Step [600/1735], Loss: 0.3595,Int100_time:388.48000597953796
Epoch [1/15], Step [700/1735], Loss: 0.3060,Int100_time:464.8700647354126
Epoch [1/15], Step [800/1735], Loss: 0.3261,Int100_time:534.8902804851532
Epoch [1/15], Step [900/1735], Loss: 0.3929,Int100_time:614.8901526927948
Epoch [1/15], Step [1000/1735], Loss: 0.3308,Int100_time:693.0157914161682
Epoch [1/15], Step [1100/1735], Loss: 0.3478,Int100_time:769.5658781528473
Epoch [1/15], Step [1200/1735], Loss: 0.3198,Int100_time:849.0171222686768
Epoch [1/15], Step [1300/1735], Loss: 0.3308,Int100_time:926.390652179718
Epoch [1/15], Step [1400/1735], 

In [17]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
confusion_matrix = torch.zeros(2, 2)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
                 
    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the test images: 89.8207368705522 %


In [18]:
print(confusion_matrix)

tensor([[35930.,  3818.],
        [ 1832., 13925.]])
